In [1]:
from huggingface_hub import hf_hub_download
import zipfile
import os


c:\Users\m.landoulsi\Desktop\pprojects\fun\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
repo_id = 'kopsahlong/DSPy_MIPRO_notebook_cache'
cache_file_path = hf_hub_download(repo_id=repo_id, filename='MIPRO_notebook_cache.zip')
compiled_program_file_path = hf_hub_download(repo_id=repo_id, filename='compiled_program.pickle')
with zipfile.ZipFile(cache_file_path, 'r') as zip_ref:
    zip_ref.extractall(".")
os.environ["DSP_NOTEBOOK_CACHEDIR"] = f"{os.getcwd()}/MIPRO_notebook_cache"


In [12]:
import openai
import dspy

### NOTE: if you'd like to run this code without a cache, you can remove these lines to configure your OPEN AI key ###
# os.environ['OPENAI_API_KEY'] = "TODO: ADD YOUR OPEN AI KEY HERE"
# openai.api_key = os.environ.get('OPENAI_API_KEY')
# openai.api_base = "https://api.openai.com/v1"

prompt_model_name = "gpt-3.5-turbo-1106"
task_model_name = "meta-llama/Llama-2-13b-chat-hf"
colbert_v2_endpoint = "http://20.102.90.50:2017/wiki17_abstracts"

prompt_model = dspy.OpenAI(model=prompt_model_name, max_tokens=150)
task_model = dspy.HFClientTGI(model=task_model_name, port=[8000], max_tokens=150,url="http://127.0.0.1")



colbertv2 = dspy.ColBERTv2(url=colbert_v2_endpoint)

dspy.settings.configure(rm=colbertv2, lm=task_model)


In [13]:
import re
from dspy.evaluate import Evaluate
from dspy.datasets import HotPotQA
from dsp.utils import EM

class ReturnRankedDocuments(dspy.Signature):
    """Given a question we are trying to answer and a list of passages, return a comma separated list of the numbers associated with each passage. These numbers should be ordered by helpfulness in answering the question, with most helpful passage number first, and the least helpful last."""
    question = dspy.InputField(desc="The question we're trying to answer.")
    context = dspy.InputField(desc="List of potentially related passages.")
    ranking = dspy.OutputField(desc="A comma separated list of numbers corresponding to passage indices, ranked in descending order by their helpfulness in answering our question.")

class RankingMultiHop(dspy.Module):
    def __init__(self, hops, num_passages_to_retrieve, max_passages_in_context):
        super().__init__()
        self.hops = hops
        self.num_passages_to_retrieve = num_passages_to_retrieve
        self.max_passages_in_context = max_passages_in_context
        self.retrieve = dspy.Retrieve(k = self.num_passages_to_retrieve)
        self.generate_query = dspy.ChainOfThought("context ,question->search_query")
        self.generate_answer = dspy.ChainOfThought("context ,question->answer")
        self.generate_ranking = dspy.ChainOfThought(ReturnRankedDocuments)

    def forward(self,question):
        context = []
        full_context = []
        top_context = []
        max_passage_num = self.max_passages_in_context
        for hop in range(self.hops):
            # Get a new query
            query = self.generate_query(context = context, question = question).search_query
            # Get new passages
            context = self.retrieve(query).passages
            # Add these new passages to the previous top context
            full_context = top_context + context
            # Get the most important indices, ranked
            most_important_indices =  self.generate_ranking(question=question, context=full_context).ranking
            indices = [int(num) for num in re.findall(r'\d+', most_important_indices)]

            if len(indices) < max_passage_num:
                indices = range(1,max_passage_num+1)

            valid_indices = [index-1 for index in indices if index-1 < len(context)]
            top_indices = sorted(valid_indices, key=lambda x: x)[:max_passage_num+1]
            most_important_context_list = [context[idx] for idx in top_indices]
            # Save the top context
            top_context = most_important_context_list

        return dspy.Prediction(context=context, answer=self.generate_answer(context = top_context , question = question).answer)

program = RankingMultiHop(hops=4, num_passages_to_retrieve=5, max_passages_in_context=5)

# Load and configure the datasets.
TRAIN_SIZE = 500
EVAL_SIZE = 500

hotpot_dataset = HotPotQA(train_seed=1, eval_seed=2023, test_size=0)
trainset = [x.with_inputs('question') for x in hotpot_dataset.train][:TRAIN_SIZE]
devset = [x.with_inputs('question') for x in hotpot_dataset.dev][:EVAL_SIZE]

# Set up metrics
NUM_THREADS = 10

metric = dspy.evaluate.answer_exact_match

kwargs = dict(num_threads=NUM_THREADS, display_progress=True)
evaluate = Evaluate(devset=devset, metric=metric, **kwargs)


  0%|          | 0/500 [08:11<?, ?it/s]


In [15]:
baseline_train_score = evaluate(program,devset=trainset)
baseline_eval_score = evaluate(program, devset=devset)

Failed to parse JSON response: Failed to parse JSON response:Failed to parse JSON response:Failed to parse JSON response:{"detail":[{"type":"missing","loc":["query","prompt"],"msg":"Field required","input":null}]}  Failed to parse JSON response:Failed to parse JSON response: Failed to parse JSON response:Failed to parse JSON response: 
Failed to parse JSON response:{"detail":[{"type":"missing","loc":["query","prompt"],"msg":"Field required","input":null}]}{"detail":[{"type":"missing","loc":["query","prompt"],"msg":"Field required","input":null}]} Failed to parse JSON response:

  0%|          | 0/500 [00:00<?, ?it/s]

{"detail":[{"type":"missing","loc":["query","prompt"],"msg":"Field required","input":null}]}  

{"detail":[{"type":"missing","loc":["query","prompt"],"msg":"Field required","input":null}]}{"detail":[{"type":"missing","loc":["query","prompt"],"msg":"Field required","input":null}]}  

{"detail":[{"type":"missing","loc":["query","prompt"],"msg":"Field required","input":null}]}{"detail":[{"type":"missing","loc":["query","prompt"],"msg":"Field required","input":null}]}
{"detail":[{"type":"missing","loc":["query","prompt"],"msg":"Field required","input":null}]}
{"detail":[{"type":"missing","loc":["query","prompt"],"msg":"Field required","input":null}]}
Failed to parse JSON response:

 Failed to parse JSON response:Failed to parse JSON response: Failed to parse JSON response: {"detail":[{"type":"missing","loc":["query","prompt"],"msg":"Field required","input":null}]}Failed to parse JSON response:Failed to parse JSON response:
 {"detail":[{"type":"missing","loc":["query","prompt"],"msg":"Field

  0%|          | 0/500 [00:55<?, ?it/s]


Exception: Received invalid JSON response from server